In [ ]:
!tar -xvf /content/qpth-0.0.15.tar.gz

qpth-0.0.15/
qpth-0.0.15/PKG-INFO
qpth-0.0.15/README.md
qpth-0.0.15/qpth/
qpth-0.0.15/qpth/__init__.py
qpth-0.0.15/qpth/qp.py
qpth-0.0.15/qpth/solvers/
qpth-0.0.15/qpth/solvers/__init__.py
qpth-0.0.15/qpth/solvers/cvxpy.py
qpth-0.0.15/qpth/solvers/pdipm/
qpth-0.0.15/qpth/solvers/pdipm/__init__.py
qpth-0.0.15/qpth/solvers/pdipm/batch.py
qpth-0.0.15/qpth/solvers/pdipm/single.py
qpth-0.0.15/qpth/solvers/pdipm/spbatch.py
qpth-0.0.15/qpth/util.py
qpth-0.0.15/qpth.egg-info/
qpth-0.0.15/qpth.egg-info/PKG-INFO
qpth-0.0.15/qpth.egg-info/SOURCES.txt
qpth-0.0.15/qpth.egg-info/dependency_links.txt
qpth-0.0.15/qpth.egg-info/requires.txt
qpth-0.0.15/qpth.egg-info/top_level.txt
qpth-0.0.15/setup.cfg
qpth-0.0.15/setup.py


In [ ]:
!pip install /content/qpth-0.0.15

Processing ./qpth-0.0.15
  Preparing metadata (setup.py) ... done
  Created wheel for qpth: filename=qpth-0.0.15-py3-none-any.whl size=15367 sha256=4104a4ae8d7e2c033dc120836e9bfb9b5833a68428829c26955892fc023ca831
  Stored in directory: /root/.cache/pip/wheels/cd/ff/f2/864ceabf2757cca1567b68ca032de25d1160c955eb71247f02
Successfully built qpth


In [ ]:
!pip install learn2learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 24.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 48.7 MB/s eta 0:00:00
  Created wheel for learn2learn: fi

In [ ]:
import os
import random

import numpy as np
import torch
from torch import nn
import torchvision as tv

import learn2learn as l2l
from learn2learn.data.transforms import FusedNWaysKShots, LoadData, RemapLabels, ConsecutiveLabels

from statistics import mean
from copy import deepcopy

In [ ]:
def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import torchvision
import matplotlib.pyplot as plt
from torch.utils.tensorboard.writer import SummaryWriter
writer = SummaryWriter('2w2sNes')

/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [ ]:
def fast_adapt(batch,
               learner,
               features,
               loss,
               adaptation_steps,
               shots,
               ways,
               device=None):

    data, labels = batch
    data, labels = data.to(device), labels.to(device)
    data = features(data)

    # Separate data into adaptation/evaluation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)

    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def main(
        ways=2,
        shots=2,
        meta_lr=0.0009,
        fast_lr=0.115,
        adapt_steps=100,
        meta_bsz=128,
        iters=400,
        cuda=1,
        seed=42,
):

    cuda = bool(cuda)

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    device = torch.device('cpu')
    if cuda and torch.cuda.device_count():
        torch.cuda.manual_seed(seed)
        device = torch.device('cuda')

    # Create Datasets
    train_dataset = l2l.vision.datasets.CIFARFS(root='~/data',
                                              transform=tv.transforms.ToTensor(),
                                              mode='train', download=True)
    valid_dataset = l2l.vision.datasets.CIFARFS(root='~/data',
                                              transform=tv.transforms.ToTensor(),
                                              mode='validation', download=True)
    test_dataset = l2l.vision.datasets.CIFARFS(root='~/data',
                                              transform=tv.transforms.ToTensor(),
                                             mode='test', download=True)
    train_dataset = l2l.data.MetaDataset(train_dataset)
    valid_dataset = l2l.data.MetaDataset(valid_dataset)
    test_dataset = l2l.data.MetaDataset(test_dataset)

    train_transforms = [
        FusedNWaysKShots(train_dataset, n=ways, k=2*shots),
        LoadData(train_dataset),
        RemapLabels(train_dataset),
        ConsecutiveLabels(train_dataset),
    ]
    train_tasks = l2l.data.Taskset(
        train_dataset,
        task_transforms=train_transforms,
        num_tasks=30000,
    )

    valid_transforms = [
        FusedNWaysKShots(valid_dataset, n=ways, k=2*shots),
        LoadData(valid_dataset),
        ConsecutiveLabels(valid_dataset),
        RemapLabels(valid_dataset),
    ]
    valid_tasks = l2l.data.Taskset(
        valid_dataset,
        task_transforms=valid_transforms,
        num_tasks=800,
    )

    test_transforms = [
        FusedNWaysKShots(test_dataset, n=ways, k=2*shots),
        LoadData(test_dataset),
        RemapLabels(test_dataset),
        ConsecutiveLabels(test_dataset),
    ]
    test_tasks = l2l.data.Taskset(
        test_dataset,
        task_transforms=test_transforms,
        num_tasks=800,
    )


    # Create model
    model = l2l.vision.models.CNN4(output_size=ways,hidden_size=64,embedding_size=64*4,)

    model.to(device)
    features = model.features
    head = model.classifier
    head=nn.DataParallel(head)
    features.to(device)
    head = l2l.algorithms.MAML(head, lr=fast_lr)
    head.to(device)


    # Setup optimization
    all_parameters = list(features.parameters()) + list(head.parameters())
    optimizer = torch.optim.NAdam(all_parameters, lr=meta_lr)
    loss = nn.CrossEntropyLoss()

    for iteration in range(iters):
        optimizer.zero_grad()
        meta_train_error = 0.0
        meta_train_accuracy = 0.0
        meta_valid_error = 0.0
        meta_valid_accuracy = 0.0
        meta_test_error = 0.0
        meta_test_accuracy = 0.0
        for task in range(meta_bsz):
            # Compute meta-training loss
            learner = head.clone()
            batch = train_tasks.sample()
            evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                               learner,
                                                               features,
                                                               loss,
                                                               adapt_steps,
                                                               shots,
                                                               ways,
                                                               device)
            evaluation_error.backward()
            meta_train_error += evaluation_error.item()
            meta_train_accuracy += evaluation_accuracy.item()

            # Compute meta-validation loss
            learner = head.clone()

            batch = valid_tasks.sample()
            evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                               learner,
                                                               features,
                                                               loss,
                                                               adapt_steps,
                                                               shots,
                                                               ways,
                                                               device)
            meta_valid_error += evaluation_error.item()
            meta_valid_accuracy += evaluation_accuracy.item()

            # Compute meta-testing loss
            learner = head.clone()
            batch = test_tasks.sample()
            evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                               learner,
                                                               features,
                                                               loss,
                                                               adapt_steps,
                                                               shots,
                                                               ways,
                                                               device)
            meta_test_error += evaluation_error.item()
            meta_test_accuracy += evaluation_accuracy.item()

        # Print some metrics
        print('\n')
        print('Iteration', iteration)
        print('Meta Train Error', meta_train_error / meta_bsz)
        print('Meta Train Accuracy', meta_train_accuracy / meta_bsz)
        print('Meta Valid Error', meta_valid_error / meta_bsz)
        print('Meta Valid Accuracy', meta_valid_accuracy / meta_bsz)
        print('Meta Test Error', meta_test_error / meta_bsz)
        print('Meta Test Accuracy', meta_test_accuracy / meta_bsz)
        writer.add_scalars('meta_train_accuracy ',
                            { 'Training' : (meta_train_accuracy/meta_bsz)*100}, iteration) #* len(batch) + iters)
        writer.add_scalars(' meta_valid_accuracy',
                            {  'Validation' : (meta_valid_accuracy/meta_bsz)*100},
                            iteration)
        writer.add_scalars('meta_test_accuracy',
                            { 'Testing' : (meta_test_accuracy / meta_bsz)*100,},
                            iteration)
    #plt.title('plot')
    #plt.plot(epochs, tr_acc, label='Meta Valid Accuracy')
    #plt.plot(tr_acc, label='Meta Test Accuracy')
    #hplt.legend(loc='best')
    #plt.xlabel('iterations')
    #plt.ylabel('train_acc')
    #plt.xticks(arange(0, 200, 40))
    #plt.show()
    #fig, ax = plt.subplots()  # Create a figure containing a single axes.
    #ax.plot(epochs, tr_acc)
        # Average the accumulated gradients and optimize
    for p in all_parameters:
      p.grad.data.mul_(1.0 / meta_bsz)
      optimizer.step()

In [ ]:
if __name__ == '__main__':
    main()

Creating CIFARFS splits


Iteration 0
Meta Train Error 0.7918859912315384
Meta Train Accuracy 0.681640625
Meta Valid Error 0.8395009242813103
Meta Valid Accuracy 0.640625
Meta Test Error 0.8474310892270296
Meta Test Accuracy 0.62109375


Iteration 1
Meta Train Error 0.7360964878171217
Meta Train Accuracy 0.685546875
Meta Valid Error 0.8586263676406816
Meta Valid Accuracy 0.63671875
Meta Test Error 0.7672527804897982
Meta Test Accuracy 0.685546875


Iteration 2
Meta Train Error 0.8373769301106222
Meta Train Accuracy 0.646484375
Meta Valid Error 0.8384480084641837
Meta Valid Accuracy 0.63671875
Meta Test Error 0.7291686274693348
Meta Test Accuracy 0.685546875


Iteration 3
Meta Train Error 0.7990355785295833
Meta Train Accuracy 0.646484375
Meta Valid Error 0.8787690062308684
Meta Valid Accuracy 0.64453125
Meta Test Error 0.820877589925658
Meta Test Accuracy 0.6484375


Iteration 4
Meta Train Error 0.774534843134461
Meta Train Accuracy 0.669921875
Meta Valid Error 0.8437547165085562
Meta 

In [ ]:
#if __name__ == '__main__':
  #  main()

In [ ]:
#f __name__ == '__main__':
    #main()

In [ ]:
#!tensorboard --logdir="/content/2w2sNes"

In [ ]:
#if __name__ == '__main__':
    #main()

In [ ]:
!zip -r /content/2w2sNes.zip /content/2w2sNes

  adding: content/2w2sNes/ (stored 0%)
  adding: content/2w2sNes/ meta_valid_accuracy_Validation/ (stored 0%)
  adding: content/2w2sNes/ meta_valid_accuracy_Validation/events.out.tfevents.1691141595.ec73c3476ef4.1748.2 (deflated 73%)
  adding: content/2w2sNes/meta_test_accuracy_Testing/ (stored 0%)
  adding: content/2w2sNes/meta_test_accuracy_Testing/events.out.tfevents.1691141595.ec73c3476ef4.1748.3 (deflated 73%)
  adding: content/2w2sNes/meta_train_accuracy _Training/ (stored 0%)
  adding: content/2w2sNes/meta_train_accuracy _Training/events.out.tfevents.1691141595.ec73c3476ef4.1748.1 (deflated 73%)
  adding: content/2w2sNes/events.out.tfevents.1691141138.ec73c3476ef4.1748.0 (deflated 9%)


In [ ]:
from google.colab import files
files.download("/content/file.zip")